Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# How to use Estimator in Azure ML

## Introduction
This tutorial shows how to use the Estimator pattern in Azure Machine Learning SDK. Estimator is a convenient object in Azure Machine Learning that wraps run configuration information to help simplify the tasks of specifying how a script is executed.


## Prerequisite:
* Understand the [architecture and terms](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture) introduced by Azure Machine Learning
* Go through the [configuration notebook](../../../configuration.ipynb) to:
    * install the AML SDK
    * create a workspace and its configuration file (`config.json`)

Let's get started. First let's import some Python libraries.

In [ ]:
import azureml.core
from azureml.core import Workspace

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

## Create an Azure ML experiment
Let's create an experiment named "estimator-test". The script runs will be recorded under this experiment in Azure.

In [ ]:
from azureml.core import Experiment

exp = Experiment(workspace=ws, name='estimator-test')

## Create or Attach existing AmlCompute
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for training your model. In this tutorial, you create `AmlCompute` as your training compute resource.

If we could not find the cluster with the given name, then we will create a new cluster here. We will create an `AmlCompute` cluster of `STANDARD_NC6` GPU VMs. This process is broken down into 3 steps:
1. create the configuration (this step is local and only takes a second)
2. create the cluster (this step will take about **20 seconds**)
3. provision the VMs to bring the cluster to the initial size (of 1 in this case). This step will take about **3-5 minutes** and is providing only sparse output in the process. Please make sure to wait until the call returns before moving to the next cell

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "cpucluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', max_nodes=4)

    # create the cluster
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    cpu_cluster.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(cpu_cluster.get_status().serialize())

Now that you have created the compute target, let's see what the workspace's `compute_targets` property returns. You should now see one entry named 'cpucluster' of type `AmlCompute`.

In [ ]:
compute_targets = ws.compute_targets
for name, ct in compute_targets.items():
    print(name, ct.type, ct.provisioning_state)

## Use a simple script
We have already created a simple "hello world" script. This is the script that we will submit through the estimator pattern. It prints a hello-world message, and if Azure ML SDK is installed, it will also logs an array of values ([Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number)).

In [ ]:
with open('./dummy_train.py', 'r') as f:
    print(f.read())

## Create A Generic Estimator

First we import the Estimator class and also a widget to visualize a run.

In [ ]:
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

The simplest estimator is to submit the current folder to the local computer. Estimator by default will attempt to use Docker-based execution. Let's turn that off for now. It then builds a conda environment locally, installs Azure ML SDK in it, and runs your script.

In [ ]:
# use a conda environment, don't use Docker, on local computer
est = Estimator(source_directory='.', compute_target='local', entry_script='dummy_train.py', use_docker=False)
run = exp.submit(est)
RunDetails(run).show()

You can also enable Docker and let estimator pick the default CPU image supplied by Azure ML for execution. You can target an AmlCompute cluster (or any other supported compute target types).

In [ ]:
# use a conda environment on default Docker image in an AmlCompute cluster
est = Estimator(source_directory='.', compute_target=cpu_cluster, entry_script='dummy_train.py', use_docker=True)
run = exp.submit(est)
RunDetails(run).show()

You can customize the conda environment by adding conda and/or pip packages.

In [ ]:
# add a conda package
est = Estimator(source_directory='.', 
                compute_target='local', 
                entry_script='dummy_train.py', 
                use_docker=False, 
                conda_packages=['scikit-learn'])
run = exp.submit(est)
RunDetails(run).show()

You can also specify a custom Docker image for exeution. In this case, you probably want to tell the system not to build a new conda environment for you. Instead, you can specify the path to an existing Python environment in the custom Docker image.

**Note**: since the below example points to the preinstalled Python environment in the miniconda3 image maintained by continuum.io on Docker Hub where Azure ML SDK is not present, the logging metric code is not triggered. But a run history record is still recorded. 

In [ ]:
# use a custom Docker image
from azureml.core.runconfig import ContainerRegistry

# this is an image available in Docker Hub
image_name = 'continuumio/miniconda3'

# you can also point to an image in a private ACR
image_registry_details = ContainerRegistry()
image_registry_details.address = "myregistry.azurecr.io"
image_registry_details.username = "username"
image_registry_details.password = "password"

# don't let the system build a new conda environment
user_managed_dependencies = True

# submit to a local Docker container. if you don't have Docker engine running locally, you can set compute_target to cpu_cluster.
est = Estimator(source_directory='.', compute_target='local', 
                entry_script='dummy_train.py',
                custom_docker_image=image_name,
                image_registry_details=image_registry_details,
                user_managed=user_managed_dependencies
                )

run = exp.submit(est)
RunDetails(run).show()

Note: if you need to cancel a run, you can follow [these instructions](https://aka.ms/aml-docs-cancel-run).

## Next Steps
Now you can proceed to explore the other types of estimators, such as TensorFlow estimator, PyTorch estimator, etc. in the sample folder.